%rsp reserved for special use:  
address|value|register|value
-|-|-|-
0x100|oxFF|%rax|0x100
0x104|0xAB|%rcx|0x1
0x108|0x13|%rdx|0x3
0x10C|0x11| |

* %rax = a register = 0x100
* 0x104 = an address = 0xAB
* $0x108 = immediate access = 0x108 
* (%rax) = parenthesis means this is an address. You go to the address and see the location = (0x100) = 0xFF
* 4(%rax) = 4 + the address of the of rax = (4 + 0x100) = (0x104) = 0xAB
* 9(%rax, %rdx) = (9 + 0x100 + 0x3) = (0x109 + 0x3) = (0x10C) = 0x11
* 260(%rcx, %rdx) = (260 + 0x1 + 0x3) = (264) in decimal = (0x108) = 0x13
* 0xFC(,%rcx,4) = 0xFC + _ + (%rcx * 4) = 0xFC + (1 * 4) = (0xFC + 4) = (0x100) = 0xFF

Memory means ( ), immediate means $, register means %


```c
void swap (long *xp, long *yp) 
{
    long t0 = *xp; /* the data in memory that stored at the address pointed by xp (*xp) move to t0 variable */
    long t1 = *yp; /* the data in memory that stored at the address pointed by yp (*yp) move to t1 variable */

    *xp = t1; /* move value of t1 to memory pointed by xp */
    *yp = t0; /* move value of t0 to memory pointed by yp */
}
```

```asm
swap:
    movq    (%rdi), %rax  # t0 = *xp
    movq    (%rsi), %rbx  # t1 = *yp
    movq    %rbx, (%rdi)  # *xp = t1
    movq    %rax, (%rsi)  # *yp = t0
    ret
```

```c
void decode1(long *xp, long *yp, long *zp);
```

```asm
# void decode1(long *xp, long *yp, long *zp)
# xp in %rdi, yp in %rsi, zp in %rdx
    # AKA
        # xp -> %rdi
        # yp -> %rsi
        # zp -> %rdx
decode1:
    movq    (%rdi), %r8
    movq    (%rsi), %rcx
    movq    (%rdx), %rax
    movq    %r8, (%rsi)
    movq    %rcx, (%rdx)
    movq    %rax, (%rdi)
    ret
```

```c
void decode1(long *xp, long *yp, long *zp){
    long x = *xp;
    long y = *yp;
    long z = *zp;

    *yp = x;
    *zp = y;
    *xp = z;
}
```

### stacks
* push operation means to add data to a stack
* pop operation means to remove data from a stack
* LAST IN, FIRST OUT

instruction|effect|description
----------|------|-----------
pushq S| R[%rsp] <- R[%rsp] -8; M[R[%rsp]] <- S|push S (quad word) 
popq D| D <- M[R[%rsp]]; R[%rsp] <- R[%rsp] + 8|pop D (quad word)

```asm
sbq $8, %rsp  # decrement stack pointer  
movq %rbs, (%rsp) # store the value of %rbp at the new stack pointer location  
```

pop %rsp is equivalent to:  

```asm
movq (%rsp), %rax # load the value of the stack pointer at the old stack pointer location  
addq $8, %rsp # increment stack pointer
```

Basically you move the "top" of the stack pointer down 8 bytes (-8), then set those 8 bytes to the new value `%rbp`   
For (POP) we just move whatever is at the stack pointer to a new variable `%rbp`, then move the stack pointer (+8)
##### We will always use %rsp for the stack pointer (top of stack).

```asm
movabsq $0x0011223344556677, %rax   # %rax = 0011223344556677
movb    $-1, %al                    # %rax = 00112233445566FF
movw    $=1, %ax                    # %rax = 001122334455FFFF
movl    $-1, %eax                   # %rax = 00000000FFFFFFFF
movq    $-1, %rax                   # %rax = FFFFFFFFFFFFFFFF

movabsq $0x0011223344556677, %rax   # %rax = 0011223344556677
movb    $0xAA, %d                   # %dl = AA
movb    %dl, %al                    # %rax = 00112233445566AA
movsbq  %dl, %rax                   # %rax = FFFFFFFFFFFFFFAA
movzbq  %dl, %rax                   # %rax = 00000000000000AA
```

Leab(l)(q) = Load Effective Address  
```asm
leaq Sec, Dst  
Src is adress mode expression  
Set Dst to address denoted by expression  
```  

long m12 (long x)
{
    return x*12;
}

```asm
leaq    (%rdi, %rdi, 2) %rax    # t <- x+x*2
salq    $2, %rax                # return t<<2
```